# **ML-AI4Econ Course**

We first download the different packages needed for this part of the course:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.inspection import DecisionBoundaryDisplay
import sklearn.datasets as data
from sklearn.linear_model import LinearRegression, QuantileRegressor, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.decomposition import PCA
import scipy.stats as stats
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, classification_report, confusion_matrix


We also set a random seed for reproducible results:

In [ ]:
np.random.seed(24)

We create some useful functions for later:

In [ ]:
def get_coef(model):
    return np.concatenate(([model.intercept_], model.coef_))

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    medae = median_absolute_error(y_true, y_pred)
    return mse, rmse, mae, r2, medae


---
### On Prediction Metrics


Before going on, we need to talk about the different metrics that we will use to evaluate whether our model is good enough.

**Mean Squared Error (MSE)**

The Mean Squared Error measures the average of the squared differences between the actual values (\(y_i\)) and the predicted values (\(\hat{y}_i\)). It is defined as:

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

where \(n\) is the number of observations.

**Root Mean Squared Error (RMSE)**

The RMSE is the square root of the MSE, providing an error measure in the same units as the target variable. The formula is:

$$
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

**Mean Absolute Error (MAE)**

The Mean Absolute Error calculates the average of the absolute differences between the actual and predicted values:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} \left|y_i - \hat{y}_i\right|
$$

This metric is less sensitive to outliers compared to the MSE.

**R-squared ($R^2$)**

The $R^2$ score represents the proportion of variance in the dependent variable that is explained by the model. It is computed as:

$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$

where $\bar{y}$ is the mean of the observed values. An $R^2$ value of 1 indicates perfect prediction, whereas a value closer to 0 indicates that the model does not explain much of the variance.

**Median Absolute Error (MedAE)**

The Median Absolute Error is the median of all absolute differences between the predicted and actual values:

$$
\text{MedAE} = \text{median} \left( \left|y_i - \hat{y}_i\right| \right)
$$

This metric provides a robust measure of the typical error, particularly useful when the data contains outliers.


These metrics are used for regression, but we can find different ones for classification tasks as well.

**Accuracy**

Accuracy measures the proportion of correct predictions out of all predictions made. It is defined as:

$$
\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}
$$

**Precision**

Precision measures the proportion of true positive predictions out of all positive predictions made by the model. It is defined as:

$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
$$

**Recall**

Recall (also known as Sensitivity or True Positive Rate) measures the proportion of actual positives that were correctly identified by the model. It is defined as:

$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
$$

**F1 Score**

The F1 Score is the harmonic mean of precision and recall, providing a balance between the two. It is defined as:

$$
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

This metric is particularly useful when the class distribution is imbalanced.

---

### Linear Regression

Now that we have seen what multivariate statistics and statistical learning theory can offer, we will go for linear models. The most important ones will be linear regression, logistic regression and LDA. We now start by first importing the data we will use throughout all the notebook, which is the "Boston Housing Prices" dataset, which has different variables related to housing characteristics in Boston and the price of the different houses in the sample.

In [ ]:
boston = data.fetch_openml(name="Boston",version=1,as_frame=True)

print("This is our basic dataset information:\n\n",boston.DESCR,"\n")
X = boston.data
print("\nThese are the original explanatory variables:\n\n",X,"\n")
y = boston.target
print("\nThese are the prices for each observation (the target variable):\n\n",y,"\n")

We can see that there are categorical variables, so we need to do some transformation! We will do so with one-hot encoding, which is basically to include binary variables for all categories but a base one (in this case, the first).

In [ ]:
encoder = OneHotEncoder(drop="first")
cat_encoded = encoder.fit_transform(X[["CHAS","RAD"]])
cat_encoded = cat_encoded.toarray()
encoded_columns = encoder.get_feature_names_out(["CHAS", "RAD"])

x_cols = list(X.columns)
x_cols.pop(3)
x_cols.pop(7)
X = X[x_cols]

cat_df = pd.DataFrame(cat_encoded, columns=encoded_columns, index=X.index)
X = pd.concat([X,cat_df],axis=1)
print("\nAfter encoding, these are the variables:\n\n",X,"\n")


Because we are interested in prediction, let us divide the dataset into train and test samples:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Now that we have the data, we focus first on the simple multivariate regression model. We try estimating different linear regression models, first starting with the most basic one.

In [ ]:
lin_reg = LinearRegression(n_jobs=2)  # We create first the object that will allow us to create the regression
lin_reg.fit(X_train,y_train) # fit the model through L2

coef = lin_reg.coef_ # get coefficients
intercept = lin_reg.intercept_ # get intercept

X_design = np.hstack([np.ones((X_train.shape[0], 1)), X_train.values]) # obtains the matrix with a column of ones
p = X_design.shape[1] # obtain number of variables
n = X_design.shape[0] # obtains number of observations

y_pred = lin_reg.predict(X=X_train) # get predictions for each X in the sample
residuals = y_train - y_pred # obtain residuals

sigma2 = np.sum(residuals**2) / (n - p) # compute variance

XtX_inv = np.linalg.inv(X_design.T @ X_design) # compute standard errors through formula
std_errors = np.sqrt(sigma2 * np.diag(XtX_inv))

t_stats = np.concatenate(([intercept], coef)) / std_errors
p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=(n - p)))

param_names = get_coef(lin_reg)
results_ols = pd.DataFrame({
    'Coefficient': np.concatenate(([intercept], coef)),
    'Std_Error': std_errors,
    't_stat': t_stats,
    'p_value': p_values
}, index=param_names) # create dataframe to visualize parameters

print("L2 Loss Model (Ordinary Least Squares) results:\n")
print(results_ols)

Let us see what happens when we change the loss function to an L1:

In [ ]:
l1_reg = QuantileRegressor(quantile=0.5, alpha=0, solver='highs')
l1_reg.fit(X_train, y_train)
coef_l1 = l1_reg.coef_
intercept_l1 = l1_reg.intercept_

results_l1 = pd.DataFrame({
    'Coefficient': np.concatenate(([intercept_l1], coef_l1))
}, index=param_names)

print("\nL1 Model (Quantile Regression at 0.5) Coefficients results:")
print(results_l1)

When we compute the L1 loss results, there is no closed formula for the coefficients, and we cannot derive the standard deviation of our coefficients. What can we do? Are we gonna be happy with just some numbers?

In this case, we can use a great statistical sampling mechanism that allows us to study variability with relatively small datasets and which is fundamental for many machine learning algorithms: we will use **bootstrap** for obtaining the variance of our coefficients.

In [ ]:
B = 1000 # number of boostrap iterations
n = X_train.shape[0] # number of obs

param_names = ['Intercept'] + list(X_train.columns)
p = len(param_names) # number of variables
coefs_bootstrap = np.zeros((B, p)) # create a matrix to save results!

for b in range(B):
    indices = np.random.choice(n, n, replace=True) # sample indices with replacement
    X_boot = X.iloc[indices] # obtain the values of the variables for the group of indices
    y_boot = y.iloc[indices]
    
    model_boot = QuantileRegressor(quantile=0.5, alpha=0, solver='highs')
    model_boot.fit(X_boot, y_boot) # we fit the model with these values
    
    coef_boot = np.concatenate(([model_boot.intercept_], model_boot.coef_)) # combine coefficients
    coefs_bootstrap[b, :] = coef_boot

std_errors_boot = np.std(coefs_bootstrap, axis=0)  # compute the standard error

l1_reg = QuantileRegressor(quantile=0.5, alpha=0, solver='highs')
l1_reg.fit(X_train, y_train)
coef_l1 = np.concatenate(([l1_reg.intercept_], l1_reg.coef_))

t_stats_l1 = coef_l1 / std_errors_boot
p_values_l1 = 2 * (1 - stats.norm.cdf(np.abs(t_stats_l1)))

results_l1_boot = pd.DataFrame({
    'Coefficient': coef_l1,
    'Bootstrap_Std_Error': std_errors_boot,
    't_stat': t_stats_l1,
    'p_value': p_values_l1
}, index=param_names)

print("=== L1 Model (Quantile Regression) with Bootstrap Inference ===")
print(results_l1_boot)

Even though we cannot plot the models, let us compare the coefficients visually:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(param_names))
width = 0.35

ax.bar(x - width/2, results_ols['Coefficient'], width, label='OLS (L2)')
ax.bar(x + width/2, results_l1['Coefficient'], width, label='L1 (Quantile)')

ax.set_xticks(x)
ax.set_xticklabels(param_names, rotation=45, ha="right")
ax.set_ylabel("Coefficient Value")
ax.set_title("Comparison of Regression Coefficients: OLS (L2) vs L1")
ax.legend()
plt.tight_layout()
plt.show()

Finally, we want to test the generalization ability of our model:

In [ ]:
y_pred_test_ols = lin_reg.predict(X_test)
y_pred_test_l1 = l1_reg.predict(X_test)

metrics_ols = evaluate_model(y_test, y_pred_test_ols)
metrics_l1 = evaluate_model(y_test, y_pred_test_l1)

metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'RMSE', 'MAE', 'R2', 'Median AE'],
    'OLS': metrics_ols,
    'L1': metrics_l1
})
print("\nEvaluation Metrics on Test Set:\n")
print(metrics_df,"\n")

# Plot Actual vs Predicted

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test_ols, label='L2', alpha=0.7)
plt.scatter(y_test, y_pred_test_l1, label='L1', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Predicted vs Actual Prices on Test Set")
plt.legend()
plt.show()

Are we able to improve these results by regularizing the models? Let us try now! In this exercise we will use different regularization techniques to compare the results with the ones obtained before. First, we estimate the new models and then we do a similar procedure.

In [ ]:
ridge_model = Ridge(alpha=1.0, random_state=42)  # L2 regularization
ridge_model.fit(X_train, y_train)

lasso_model = Lasso(alpha=0.1, random_state=42, max_iter=10000)  # L1 regularization
lasso_model.fit(X_train, y_train)

elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42, max_iter=10000)
elastic_model.fit(X_train, y_train)

coef_ols = get_coef(lin_reg)
coef_ridge = get_coef(ridge_model)
coef_lasso = get_coef(lasso_model)
coef_elastic = get_coef(elastic_model)

coef_df = pd.DataFrame({
    'OLS': coef_ols,
    'Ridge': coef_ridge,
    'Lasso': coef_lasso,
    'ElasticNet': coef_elastic
}, index=param_names)

print("\nCoefficients from Different Models:\n")
print(coef_df)

Let us take a look at the coefficients through a visual comparison:

In [ ]:
x = np.arange(len(param_names))  # one position for each parameter
width = 0.2  # width for each bar

fig, ax = plt.subplots(figsize=(14, 7))
ax.bar(x - 1.5*width, coef_df['OLS'], width, label='OLS')
ax.bar(x - 0.5*width, coef_df['Ridge'], width, label='Ridge')
ax.bar(x + 0.5*width, coef_df['Lasso'], width, label='Lasso')
ax.bar(x + 1.5*width, coef_df['ElasticNet'], width, label='ElasticNet')

ax.set_xticks(x)
ax.set_xticklabels(param_names, rotation=45, ha='right')
ax.set_ylabel("Coefficient Value")
ax.set_title("Comparison of Regression Coefficients")
ax.legend()
plt.tight_layout()
plt.show()

Finally, how did our regularized models do?

In [ ]:
y_pred_ols = lin_reg.predict(X_test)
y_pred_ridge = ridge_model.predict(X_test)
y_pred_lasso = lasso_model.predict(X_test)
y_pred_elastic = elastic_model.predict(X_test)

metrics_ols = evaluate_model(y_test, y_pred_ols)
metrics_ridge = evaluate_model(y_test, y_pred_ridge)
metrics_lasso = evaluate_model(y_test, y_pred_lasso)
metrics_elastic = evaluate_model(y_test, y_pred_elastic)

metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'RMSE', 'MAE', 'R2', 'Median AE'],
    'OLS': metrics_ols,
    'Ridge': metrics_ridge,
    'Lasso': metrics_lasso,
    'ElasticNet': metrics_elastic
})
print("\n=== Evaluation Metrics on Test Set ===")
print(metrics_df,"\n")

# Plot of Actual vs Predicted

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_ols, label='OLS', alpha=0.7)
plt.scatter(y_test, y_pred_ridge, label='Ridge', alpha=0.7)
plt.scatter(y_test, y_pred_lasso, label='Lasso', alpha=0.7)
plt.scatter(y_test, y_pred_elastic, label='ElasticNet', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Predicted vs Actual Prices on Test Set")
plt.legend()
plt.show()

Is there anything we can still do to get a good generalization, apart from changing the loss function we estimate the model with? Yes. We can still try to find a better model by using a technique called **cross-validation** for **hyperparameter tuning**. This will allow us to obtain a "best" model for our regularized models and assess whether our generalization ability was good enouigh even before testing the data.

---

### On Cross-validation

Cross validation involves partitioning the dataset into several folds. In each iteration, one fold is used as the test set while the remaining folds are used for training. This process is repeated until each fold has served as the test set exactly once. Some common types of cross-validation are the following:

- **K-Fold Cross Validation:**  
  The data is divided into K equal parts (folds). The model is trained K times, each time leaving out one of the folds as the test set. The performance is then averaged over the K iterations.

- **Leave-One-Out Cross Validation (LOOCV):**  
  A special case of K-Fold where K is equal to the number of observations. Each observation is used once as the test set, while the remaining observations form the training set.

- **Stratified K-Fold Cross Validation:**  
  Used mainly for classification, this method ensures that each fold has approximately the same percentage of samples of each target class as the complete dataset.

Cross validation provides a more reliable estimate of model performance compared to a single train/test split, and also allows choosing between hyperparameters. It reduces the risk of overfitting and ensures that the model's evaluation is not dependent on one particular split of the data. The practical process would be the following:

1. Divide the dataset into K folds.
2. For each fold:
   - Train the model on the remaining K-1 folds.
   - Test the model on the current fold.
3. Calculate the performance metric for each fold.
4. Average the metrics across all folds to obtain the final performance measure.
5. If the model has hyperparameters

---

In [ ]:
ridge_metrics = evaluate_model(y_test, y_pred_ridge)
lasso_metrics = evaluate_model(y_test, y_pred_lasso)

ridge_params = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
lasso_params = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]}

ridge_cv = GridSearchCV(Ridge(random_state=42), ridge_params, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train, y_train)
best_ridge = ridge_cv.best_estimator_

mean_train_score = -ridge_cv.cv_results_["mean_test_score"][ridge_cv.best_index_]
print("Mean Training MSE from CV (Ridge): {:.4f}".format(mean_train_score),"\n\n")

lasso_cv = GridSearchCV(Lasso(random_state=42), lasso_params, cv=5, scoring='neg_mean_squared_error')
lasso_cv.fit(X_train, y_train)
best_lasso = lasso_cv.best_estimator_

mean_train_score = -lasso_cv.cv_results_["mean_test_score"][lasso_cv.best_index_]
print("Mean Training MSE from CV (Lasso): {:.4f}".format(mean_train_score),"\n\n")

best_ridge_pred = best_ridge.predict(X_test)
best_lasso_pred = best_lasso.predict(X_test)

best_ridge_metrics = evaluate_model(y_test, best_ridge_pred)
best_lasso_metrics = evaluate_model(y_test, best_lasso_pred)

models = [
    'Ridge (default)',
    'Lasso (default)',
    'Ridge (CV tuned)',
    'Lasso (CV tuned)'
]

mse_values = [
    ridge_metrics[0],
    lasso_metrics[0],
    best_ridge_metrics[0],
    best_lasso_metrics[0],
]

mae_values = [
    ridge_metrics[2],
    lasso_metrics[2],
    best_ridge_metrics[2],
    best_lasso_metrics[2]
]

r2_values = [
    ridge_metrics[3],
    lasso_metrics[3],
    best_ridge_metrics[3],
    best_lasso_metrics[3]
]

results_df = pd.DataFrame({
    'Model': models,
    'MSE': mse_values,
    'MAE': mae_values,
    'R2': r2_values
})

print("Test Set Performance Comparison:\n")
print(results_df)

Graphically, we can spot the changes that happened here:

In [ ]:
coef_ridge_cv = get_coef(best_ridge)
coef_lasso_cv = get_coef(best_lasso)

coef_df = pd.DataFrame({
    'Ridge (default)': coef_ridge,
    'Lasso (default)': coef_lasso,
    'Ridge (CV tuned)': coef_ridge_cv,
    'Lasso (CV tuned)': coef_lasso_cv
}, index=['Intercept'] + list(X_train.columns))

coef_df.plot(kind='bar', figsize=(14,7))
plt.ylabel("Coefficient Value")
plt.title("Comparison of Coefficients for Different Models")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

And we can also take a look at the predictions:

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_ridge, label='Ridge', alpha=0.7)
plt.scatter(y_test, y_pred_lasso, label='Lasso', alpha=0.7)
plt.scatter(y_test, best_ridge_pred, label='Ridge CV', alpha=0.7)
plt.scatter(y_test, best_lasso_pred, label='Lasso CV', alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Predicted vs Actual Prices on Test Set")
plt.legend()
plt.show()

We have seen a lot of stuff! In fact, we have seen techniques for **evaluation**, **model selection**, **data transformation** and **regularization**. Therefore, these are now assumed to be known, and we will apply them regularly without the need of comparing different possibilities from now on. We can now continue to linear models for classification.

### Logistic Regression

Now, we want a linear model that can work for classification, given that the linear regression model is not suited for this purpose. One of the best optios is the logistic regression, which basically is a generalized linear model that allows to classify different observations based on the probability given by the model to each of the possible classes or categories. In this case, we will not use the "Boston Housing Prices" dataset, but the "Iris" dataset.

In [ ]:
iris = data.fetch_openml(name='iris', version=1, as_frame=True)
iris_desc = load_iris()
X_iris = iris.data
y_iris = iris.target
target_names = iris.target_names

print("This is our basic dataset information:\n\n",iris_desc.DESCR,"\n")
print("Features:\n\n")
print(X_iris)
print("\n\nTarget:\n\n")
print(y_iris)

We now divide train and test data:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)

Now, we estimate our logistic regression model with our train data and evaluate the performance of our model:

In [ ]:
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

# Evaluate the model using classification report and confusion matrix

print("\nClassification Report:\n\n")
print(classification_report(y_test, y_pred),"\n")
print("\nConfusion Matrix:\n\n")
print(confusion_matrix(y_test, y_pred))

We can take a look at how we are assigning each observation to a group.

In [ ]:
probabilities = logreg.predict_proba(X_test)
prob_df = pd.DataFrame(probabilities, columns=[f"Probability(Class {i})" for i in range(probabilities.shape[1])])
print("Predicted class probabilities for some test observations:\n")
print(prob_df.head())

### Linear Discriminant Analysis

A different approach would be to use linear discriminant analysis. In this case, we try to infer some Bayesian decision boundaries from the data among categories to classify different observations, assuming normality, and we play with Bayes Theorem in order to provide a posterior probability of pertaining to one class for each observation.

In [ ]:
X_plot = X_iris.iloc[:, 2:4]
X_train, X_test, y_train, y_test = train_test_split(X_plot, y_iris, test_size=0.3, random_state=42)

In [ ]:
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()

lda.fit(X_train, y_train)
qda.fit(X_train, y_train)

y_pred_lda = lda.predict(X_test)
y_pred_qda = qda.predict(X_test)

# Evaluate the model using classification report and confusion matrix

print("\nClassification Report for LDA:\n\n")
print(classification_report(y_test, y_pred_lda),"\n")
print("\nConfusion Matrix for LDA:\n\n")
print(confusion_matrix(y_test, y_pred_lda))

print("\nClassification Report for QDA:\n\n")
print(classification_report(y_test, y_pred_qda),"\n")
print("\nConfusion Matrix for QDA:\n\n")
print(confusion_matrix(y_test, y_pred_qda))

Now, we can observe some observations' probabilities of pertaining to one group or another:

In [ ]:
proba_lda = lda.predict_proba(X_test)
proba_qda = qda.predict_proba(X_test)


print("Predicted class probabilities (LDA) for first 5 test samples:")
print(proba_lda[:5])
print("\nPredicted class probabilities (QDA) for first 5 test samples:")
print(proba_qda[:5])

For LDA and QDA, the best idea is to visualize what the decision boundary looks like:

In [ ]:
iris = data.load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

X_plot = X_iris.iloc[:, 2:4]

X_train, X_test, y_train, y_test = train_test_split(X_plot, y, test_size=0.3, random_state=42)

lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()

lda.fit(X_train, y_train)
qda.fit(X_train, y_train)

x_min, x_max = X_train.iloc[:, 0].min() - 1, X_train.iloc[:, 0].max() + 1
y_min, y_max = X_train.iloc[:, 1].min() - 1, X_train.iloc[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))
grid_points = np.c_[xx.ravel(), yy.ravel()]

Z_lda = lda.predict(grid_points).reshape(xx.shape)
Z_qda = qda.predict(grid_points).reshape(xx.shape)

fig, axs = plt.subplots(1, 2, figsize=(14, 6), constrained_layout=True)
cmap = plt.cm.Paired

axs[0].contourf(xx, yy, Z_lda, alpha=0.3, cmap=cmap)
sc0 = axs[0].scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=y_test, edgecolor='k', cmap=cmap, marker='o', s=50)
axs[0].set_xlabel("Petal length (cm)")
axs[0].set_ylabel("Petal width (cm)")
axs[0].set_title("LDA Decision Boundaries (Test Data)")
handles0, _ = sc0.legend_elements()
axs[0].legend(handles=handles0, labels=list(target_names), title="Classes")

axs[1].contourf(xx, yy, Z_qda, alpha=0.3, cmap=cmap)
sc1 = axs[1].scatter(X_test.iloc[:, 0], X_test.iloc[:, 1], c=y_test, edgecolor='k', cmap=cmap, marker='o', s=50)
axs[1].set_xlabel("Petal length (cm)")
axs[1].set_ylabel("Petal width (cm)")
axs[1].set_title("QDA Decision Boundaries (Test Data)")
handles1, _ = sc1.legend_elements()
axs[1].legend(handles=handles1, labels=list(target_names), title="Classes")

plt.show()